<a href="https://colab.research.google.com/github/noattran/LSTM-INC/blob/main/LSTM-myINC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator

In [2]:
path = r'dataset/file_data_1.csv'
data = pd.read_csv(path)
data.head()

,flow_duration,Header_Length,Protocol Type,Duration,Rate,Srate,Drate,fin_flag_number,syn_flag_number,rst_flag_number,...,Std,Tot size,IAT,Number,Magnitue,Radius,Covariance,Variance,Weight,label
0,37.887028,1687747.1,9.3,63.5,34.048221,34.048221,0.0,0.0,0.0,0.0,...,332.122451,817.5,5.235219e-03,5.5,24.333355,469.692075,2.092362e+05,0.9,38.5,Benign
1,48.272384,128041.4,10.4,97.8,39966.184035,39966.184035,0.0,0.0,0.0,0.0,...,694.907070,636.4,1.665200e+08,13.5,36.393289,983.968335,4.842123e+05,1.0,244.6,Benign
2,48.095663,101026.4,8.8,137.7,13.947935,13.947935,0.0,0.0,0.0,0.0,...,1302.117190,919.3,2.724581e-02,5.5,47.214170,1841.471789,2.063959e+06,0.9,38.5,Benign
3,46.342407,1301464.8,4.8,168.3,34.223130,34.223130,0.0,0.0,0.0,0.0,...,0.428562,64.8,3.804951e-02,5.5,11.472815,0.606078,9.315556e-01,0.2,38.5,Benign
4,88.807750,5290348.6,7.1,63.6,61.443831,61.443831,0.0,0.0,0.0,0.0,...,101.938534,1227.3,1.833391e-03,5.5,54.184531,144.162858,5.276998e+04,0.2,38.5,Benign


In [3]:
# Chuẩn bị dữ liệu
X, y = data.drop('label', axis=1), data['label']
scaler = StandardScaler()
le = LabelEncoder()
X_scaled = scaler.fit_transform(X)
y_encoded = le.fit_transform(y)

In [4]:
X.shape

(466866, 46)

In [5]:
# Chia dữ liệu thành tập huấn luyện và kiểm tra
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_encoded, test_size=0.2, random_state=42)

In [6]:
len(le.classes_)

8

In [7]:
from tensorflow.keras.utils import to_categorical
# Chuyển y thành dạng one-hot để phân loại
y_train = to_categorical(y_train.squeeze())
y_test = to_categorical(y_test.squeeze())

n_input = 100
n_features = X.shape[1]
train_generator = TimeseriesGenerator(X_train, y_train, length=n_input, batch_size=32)
test_generator = TimeseriesGenerator(X_test, y_test, length=n_input, batch_size=32)
model = Sequential()
model.add(LSTM(128, activation='relu', input_shape=(n_input, n_features)))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(len(le.classes_), activation='softmax'))

In [8]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 128)               89600     
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense (Dense)               (None, 64)                8256      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dropout_2 (Dropout)         (None, 32)                0         
                                                                 
 dense_2 (Dense)             (None, 8)                 2

In [ ]:
#huấn luyện
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(train_generator, epochs=10, validation_data=test_generator)
# Đánh giá mô hình trên tập kiểm tra
loss, accuracy = model.evaluate(test_generator)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


11669/11669 ━━━━━━━━━━━━━━━━━━━━ 201s 17ms/step - accuracy: 0.7246 - loss: 230.8966 - val_accuracy: 0.7259 - val_loss: 0.8806
Epoch 2/10
11669/11669 ━━━━━━━━━━━━━━━━━━━━ 190s 16ms/step - accuracy: 0.7280 - loss: 123.3991 - val_accuracy: 0.7259 - val_loss: 0.8804
Epoch 3/10
11669/11669 ━━━━━━━━━━━━━━━━━━━━ 202s 16ms/step - accuracy: 0.7295 - loss: 0.8719 - val_accuracy: 0.7259 - val_loss: 0.8807
Epoch 4/10
11669/11669 ━━━━━━━━━━━━━━━━━━━━ 201s 16ms/step - accuracy: 0.7292 - loss: 0.8748 - val_accuracy: 0.7258 - val_loss: 0.8809
Epoch 5/10
11669/11669 ━━━━━━━━━━━━━━━━━━━━ 190s 16ms/step - accuracy: 0.7273 - loss: 1.2466 - val_accuracy: 0.7259 - val_loss: 0.8807
Epoch 6/10
11669/11669 ━━━━━━━━━━━━━━━━━━━━ 202s 16ms/step - accuracy: 0.7293 - loss: 0.8797 - val_accuracy: 0.7259 - val_loss: 0.8804
Epoch 7/10
11669/11669 ━━━━━━━━━━━━━━━━━━━━ 203s 16ms/step - accuracy: 0.7286 - loss: 2.9448 - val_accuracy: 0.7259 - val_loss: 0.8804
Epoch 8/10
11669/11669 ━━━━━━━━━━━━━━━━━━━━ 202s 16ms/step - a

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
#tính RMSE và MSE
y_pred = model.predict(test_generator)
# predictions = np.argmax(predictions, axis=1)
#tính toán các metrics
y_test_trimmed = y_test[n_input:]
print("============================================")
mse = mean_squared_error(y_test_trimmed, y_pred)
mae = mean_absolute_error(y_test_trimmed, y_pred)
r2 = r2_score(y_test_trimmed, y_pred)
rmse = np.sqrt(mse)
print("Các chỉ số của dữ liệu: ")
print("Mean Absolute Error (MAE):", mae)
print("Mean Squared Error (MSE):", mse)
print("Root Mean Squared Error (RMSE):", rmse)
print("============================================")

2915/2915 ━━━━━━━━━━━━━━━━━━━━ 18s 6ms/step
Các chỉ số của dữ liệu: 
Mean Absolute Error (MAE): 0.10891099167471403
Mean Squared Error (MSE): 0.05484419841514618
Root Mean Squared Error (RMSE): 0.23418838232317626


In [ ]:
import time
start_time = time.time()
model.fit(train_generator, epochs=1, validation_data=test_generator)
end_time = time.time()
print(f"Thời gian huấn luyện: {end_time - start_time} giây")

11669/11669 ━━━━━━━━━━━━━━━━━━━━ 192s 16ms/step - accuracy: 0.7207 - loss: 40364.2344 - val_accuracy: 0.7259 - val_loss: 0.8805
Thời gian huấn luyện: 202.0703990459442 giây


In [14]:
# Incremental Learning
import time
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
num_samples_total = X_train.shape[0]
batch_size = 256
epochs_per_batch = 5
time_all = []
num_batches = num_samples_total // batch_size
for batch_idx in range(num_batches):
    # Chia dữ liệu theo từng batch
    start_idx = batch_idx * batch_size
    end_idx = start_idx + batch_size
    X_batch, Y_batch = X_train[start_idx:end_idx], y_train[start_idx:end_idx]
    train_generator = TimeseriesGenerator(X_batch, Y_batch, length=n_input, batch_size=32)
    
    # Huấn luyện trên batch hiện tại
    print(f"Training on batch {batch_idx + 1}/{num_batches}...")
    start = time.time()
    model.fit(train_generator, epochs=epochs_per_batch, verbose=1, batch_size=batch_size)
    end = time.time()
    time_all.append(end - start)



Training on batch 1/1458...
Epoch 1/5
5/5 [==============================] - 3s 315ms/step - loss: 0.9725 - accuracy: 0.7051
Epoch 2/5
5/5 [==============================] - 2s 303ms/step - loss: 0.9016 - accuracy: 0.7051
Epoch 3/5
5/5 [==============================] - 2s 304ms/step - loss: 0.8848 - accuracy: 0.7051
Epoch 4/5
5/5 [==============================] - 2s 314ms/step - loss: 0.8782 - accuracy: 0.7051
Epoch 5/5
5/5 [==============================] - 2s 304ms/step - loss: 0.8552 - accuracy: 0.7051
Training on batch 2/1458...
Epoch 1/5
5/5 [==============================] - 2s 311ms/step - loss: 0.8960 - accuracy: 0.7115
Epoch 2/5
5/5 [==============================] - 2s 308ms/step - loss: 0.8871 - accuracy: 0.7115
Epoch 3/5
5/5 [==============================] - 2s 304ms/step - loss: 0.8850 - accuracy: 0.7051
Epoch 4/5
5/5 [==============================] - 2s 298ms/step - loss: 0.8643 - accuracy: 0.7115
Epoch 5/5
5/5 [==============================] - 2s 302ms/step - loss: 

In [15]:
# Đánh giá mô hình
print("\nFinal Evaluation:")
test_generator = TimeseriesGenerator(X_test, y_test, length=n_input, batch_size=32)
loss, mae = model.evaluate(test_generator, verbose=1)
print(f"Loss: {loss}, MAE: {mae}")
print('Time: ', np.sum(time_all))


Final Evaluation:
2915/2915 [==============================] - 149s 51ms/step - loss: 0.8807 - accuracy: 0.7259
Loss: 0.8806945085525513, MAE: 0.7258614301681519
Time:  11354.406195402145
